# Attempting to reproduce factor loadings from 5-factor model Karlsson et al using Harmony

Karlsson P, Larm P, Svensson J, Raninen J (2022) The factor structure of the Strength and Difficulties Questionnaire in a national sample of Swedish adolescents: Comparing 3 and 5-factor models. PLOS ONE 17(3): e0265481. https://doi.org/10.1371/journal.pone.0265481
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0265481

![Karlsson P, Larm P, Svensson J, Raninen J (2022) The factor structure of the Strength and Difficulties Questionnaire in a national sample of Swedish adolescents: Comparing 3 and 5-factor models. PLOS ONE 17(3): e0265481. https://doi.org/10.1371/journal.pone.0265481](journal.pone.0265481.t001.PNG "Karlsson P, Larm P, Svensson J, Raninen J (2022) The factor structure of the Strength and Difficulties Questionnaire in a national sample of Swedish adolescents: Comparing 3 and 5-factor models. PLOS ONE 17(3): e0265481. https://doi.org/10.1371/journal.pone.0265481")


Questionnaire used: https://www.sdqinfo.org/py/sdqinfo/b3.py?language=Englishqz(UK) - chose SDQ_English(UK)_s11-17full.pdf

In [1]:
import harmony

2023-07-13 09:42:16.134860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 09:42:21.562119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-13 09:42:21.562175: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-13 09:42:22.020479: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-13 09:42:42.559848: W tensorflow/stream_executor/platform/de

In [2]:
from harmony import load_instruments_from_local_file

In [3]:
instruments = load_instruments_from_local_file("SDQ_English(UK)_s11-17full.pdf")

Preparing data for Tika
Calling Tika
Got response from Tika
Parsed response from Tika


/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/harmony/parsing/text_extraction/smart_document_parser.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred.rename(columns={"preceding_bullet_value": "question_no"}, inplace=True)


Excluding question Date of Birth
Excluding question Yes- minor difficulties
Excluding question If you have answered "Yes", please answer the following questions about these difficulties
Excluding question Less than a month
Excluding question A great deal
Excluding question CLASSROOM LEARNING
Excluding question Today's Date
Excluding question Robert Goodman,
Excluding question Yes- definite difficulties
Excluding question Yes- severe difficulties
Excluding question A great deal
Excluding question A great deal
Excluding question Female Date of Birth
Excluding question Not True Somewhat True Certainly True
Excluding question No Yes- minor difficulties
Excluding question If you have answered "Yes", please answer the following questions about these difficulties
Excluding question Less than a month
Excluding question months Over a year
Excluding question Not at all Only a little Quite a lot A great deal
Excluding question Not at all Your Signature
Excluding question Today's Date
Excluding qu

In [8]:
len(instruments)

1

In [10]:
# import json
# import pickle as pkl
# with open("sdq.txt", "w", encoding="utf-8") as f:
#     f.write(instruments[0].json())

In [3]:
from harmony.schemas.requests.text import Instrument

In [5]:
import pickle as pkl
with open("sdq.txt", "r", encoding="utf-8") as f:
    instruments = [Instrument.parse_raw(f.read())]

In [6]:
instruments

[Instrument(file_id='3f4a0966189645fc97c0b72e54263e86', instrument_id='3f4a0966189645fc97c0b72e54263e86_0', instrument_name='Untitled file', file_name='Untitled file', file_type=<FileType.pdf: 'pdf'>, file_section='', study=None, sweep=None, metadata=None, language=<Language.English: 'en'>, questions=[Question(question_no='1', question_intro='', question_text='It would help us if you answered all items as best you can even if you are not absolutely certain or the item seems daft', options=[], source_page=0, instrument_id=None, instrument_name=None, topics_auto=None, nearest_match_from_mhc_auto=None), Question(question_no='2', question_intro='', question_text='Please give your answers on the basis of how things have been for you over the last six months.', options=[], source_page=0, instrument_id=None, instrument_name=None, topics_auto=None, nearest_match_from_mhc_auto=None), Question(question_no='4', question_intro='', question_text='I care about their feelings', options=[], source_pag

In [62]:
seen = set()
deduplicated_questions = []
for question in instruments[0].questions:
    # remove some of the questions that Harmony should not have picked up
    if "item seems daft" in question.question_text or "double sided template" in question.question_text \
        or "Your Signature" in question.question_text or "LEISURE ACTIVITIES" in question.question_text \
    or "Please give your answers on the basis" in question.question_text \
    or "Thank you very much for your help" in question.question_text:
        continue
    if question.question_text not in seen:
        deduplicated_questions.append(question)
    seen.add(question.question_text)

In [54]:
instruments = [Instrument(questions=deduplicated_questions)]

In [55]:
from harmony import match_instruments

In [56]:
all_questions, similarity, query_similarity, new_vectors_dict = match_instruments(instruments)

In [57]:
import pickle as pkl
with open("similarity.pkl", "wb") as f:
    pkl.dump(similarity, f)

In [58]:
threshold = 0.5

In [59]:
groups = []
for q1 in range(similarity.shape[0]):
    for q2 in range(similarity.shape[1]):
        s = similarity[q1][q2]
        if s > threshold:
            is_found = False
            for group in groups:
                if q1 in group or q2 in group:
                    group.add(q1)
                    group.add(q2)
                    is_found = True
            if not is_found:
                groups.append({q1, q2})                

# Print out Harmony's groups

In [61]:
for idx, g in enumerate(groups):
    print (f"Group {idx}")
    for q_id in g:
        print ("\t" + all_questions[q_id].question_no + " " + all_questions[q_id].question_text)

Group 0
	4 I care about their feelings
Group 1
	5 I am restless, I cannot stay still for long
Group 2
	6 I get a lot of headaches, stomach-aches or sickness
Group 3
	7 I usually share with others (food, games, pens etc
	25 I take things that are not mine from home, school or elsewhere
Group 4
	42 Do the difficulties make it harder for those around you (family, friends, teachers, etc.
	8 I get very angry and often lose my temper
	12 I am helpful if someone is hurt, upset or feeling ill
	16 I am often unhappy, down-hearted or tearful
	19 I easily lose confidence
	31 Overall, do you think that you have difficulties in one or more of the following areas
	32 emotions, concentration, behaviour or being able to get on with other people?
	35 How long have these difficulties been present?
	37 Do the difficulties upset or distress you?
	39 Do the difficulties interfere with your everyday life in the following areas?
Group 5
	9 I generally play alone or keep to myself
Group 6
	10 I usually do as 